In [3]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
def replace_pattern_for_script(text):
    pattern = r'\([^)]+\)/\(([^)]+)\)'
    replaced_text = re.sub(pattern, r'\1', text)
    replaced_text = re.sub(r'[a-z]\/', '', replaced_text)
    replaced_text = replaced_text.replace('/', '')
    replaced_text = replaced_text.replace('  ', ' ')
    return replaced_text


#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            category = line['typeInfo']['category']
            subcategory = line['typeInfo']['subcategory']
            inputType = line['typeInfo']['inputType']
            texts = line['texts']
            new_texts = []
            for text in texts:
                new_text = replace_pattern_for_script(text)
                new_texts.append(new_text)
            #### data preprocess end 
            
            
            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = f'### 회의 정보: \n{category} {subcategory} {inputType} \n\n### 회의 내용: \n' + '\n'.join(new_texts)
            data['text'] = text
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train:   0%|          | 0/15 [00:00<?, ?it/s]

valid: 100%|██████████| 8/8 [00:00<00:00, 125.78it/s]
